# Generator of Address for Kleros NEO
This notebook will generate the list of addresses that can participate in Kleros Neo (in Arbitrum)
To be in the whitelist you should meet any of this 2 requirements:
1. Has staked at least 1 PNK in Mainnet or Gnosis.
2. Currently Registered in PoH (those who has registered but with a non-active profile   are out).
3. Have participated in the Testnet tests of Court v2

The snapshot to filter these wallets is at Thursday, March 3, 2024 4:36:10 UTC (timestamp: 1709440570)

In [32]:
import csv

from app.utils.subgraph import KlerosBoardSubgraph, PoHSubgrpah

In [33]:
snapshot_timestamp = 1709440570

In [34]:
from typing import Set


def set_to_csv(data: Set, filename: str):
    with open('lists/' + filename, 'w', newline='\n') as file:
        wr = csv.writer(file, quoting=csv.QUOTE_NONE, lineterminator=',\n')
        for item in sorted(data):
            wr.writerow([item])

## Kleros Jurors
In this section we are going to get the list of all the jurors no matter if they have a stake now or not.

In [35]:
kb_eth = KlerosBoardSubgraph('mainnet')
kb_gno = KlerosBoardSubgraph('gnosis')

# Get the list of all the StakeSet events for both chains
allStakes_eth = kb_eth.getAllStakeSets()
allStakes_gno = kb_gno.getAllStakeSets()

# generate a set with all the addresses in the events.
# Must to be a set to remove duplicates.
jurors_eth_set = set(
    [stake['address'] for stake in allStakes_eth if stake['timestamp'] <= snapshot_timestamp]
)
jurors_gno_set = set(
    [stake['address'] for stake in allStakes_gno if stake['timestamp'] <= snapshot_timestamp]
)
print(f'There are {len(jurors_eth_set)} jurors in mainnet the set and {len(jurors_gno_set)} in the gnosis set.')


There are 2190 jurors in mainnet the set and 221 in the gnosis set.


In [36]:
jurors_set = jurors_eth_set.union(jurors_gno_set)
print(f'There are {len(jurors_set)} jurors in the set considering Ethereum and Gnosis')
# export list to csv
set_to_csv(jurors_set, 'v1-jurors.csv')

There are 2306 jurors in the set considering Ethereum and Gnosis


# Poh Profiles
Get the list of PoH profiles that are active at date March 13th (meaning submissionTime bigger than today - 2years = 1646282170)


In [37]:
poh = PoHSubgrpah()
submissionTime = 1646282170  # snapshot timestamp - 2 years
submissions = poh.getAllSubmissions(submissionTime)
profiles_set = set([submission['id'] for submission in submissions])

In [38]:
print(f'There are {len(profiles_set)} profiles in the set')

# export list to csv
set_to_csv(profiles_set, 'poh-profiles.csv')

There are 1452 profiles in the set


# V2 List

In [39]:
v2_jurors_set = set()
with open('lists/v2-neo-early-users-from-v2-testnets.csv', newline='') as csvfile:
    juror_reader = csv.reader(csvfile)
    for v2_juror in list(juror_reader):
        v2_jurors_set.add(v2_juror[0])  # Due to the trailing comma, we need to keep just the first element

print(f'There are {len(v2_jurors_set)} jurors in the v2 testnet set')


There are 161 jurors in the v2 testnet set


# NEO list
Then we merge the three sets in order to get the full list of participants in the Kleros v2NEO Court

In [40]:
v2neo_set = jurors_set.union(profiles_set).union(v2_jurors_set)

In [41]:
print(f'There are {len(v2neo_set)} profiles in the set')

# export list to csv
set_to_csv(v2neo_set, 'v2-neo-early-users-final.csv')

There are 3859 profiles in the set
